# Clustering K-Means com Distância Euclidiana
## Análise de Movimento baseada em Acelerômetro e Giroscópio

Este notebook implementa clustering K-means para classificação de movimento em 3 categorias:
- 🔴 **Muito baixo movimento (parado)**: Baixa variação - sono/inatividade
- 🔵 **Baixo movimento**: Variação média - movimento leve
- 🟢 **Alto movimento**: Alta variação - movimento intenso

**Metodologia:**
- Janelas temporais SEM sobreposição (10 pontos por janela)
- Ordenação por variação (std) do acelerômetro
- Features: [std_accel, mag_accel, std_gyro, mag_gyro]
- Dados com downsampling (50% dos pontos originais)

## 1. Importações e Configurações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime

# Configurações
N_CLUSTERS = 3          # Número de clusters (muito baixo, baixo, alto movimento)
WINDOW_SIZE = 10        # Tamanho da janela temporal
PESSOA_INICIAL = 11     # Primeira pessoa do dataset com downsampling
PESSOA_FINAL = 38       # Última pessoa do dataset

print("Configurações carregadas!")
print(f"- Clusters: {N_CLUSTERS}")
print(f"- Tamanho da janela: {WINDOW_SIZE} pontos")
print(f"- Pessoas: {PESSOA_INICIAL} a {PESSOA_FINAL}")

## 2. Funções de Carregamento de Dados

In [ ]:
def carregar_dados_pessoa_downsampled(pessoa_id):
    """
    Carrega dados de acelerômetro e giroscópio de uma pessoa específica (dados com downsampling)
    
    Args:
        pessoa_id: ID da pessoa (número)
    
    Returns:
        tuple: (DataFrame acelerômetro, DataFrame giroscópio)
    """
    # Caminhos dos arquivos com downsampling
    accel_file = f'../Downsampling_data/ds_acelerometro/ds_acelerometro_{pessoa_id}.csv'
    gyro_file = f'../Downsampling_data/ds_giroscopio/ds_giroscopio_{pessoa_id}.csv'
    
    # Carregar acelerômetro
    df_accel = pd.read_csv(accel_file)
    df_accel['timestamp'] = pd.to_datetime(df_accel['timestamp'])
    
    # Carregar giroscópio
    df_gyro = pd.read_csv(gyro_file)
    df_gyro['timestamp'] = pd.to_datetime(df_gyro['timestamp'])
    
    return df_accel, df_gyro

## 3. Sincronização de Dados

In [ ]:
def sincronizar_dados(df_accel, df_gyro):
    """
    Sincroniza dados de acelerômetro e giroscópio baseado em timestamps
    
    Args:
        df_accel: DataFrame com dados do acelerômetro
        df_gyro: DataFrame com dados do giroscópio
    
    Returns:
        DataFrame: Dados sincronizados
    """
    # Renomear colunas do giroscópio para evitar conflito
    df_gyro = df_gyro.rename(columns={'x': 'gx', 'y': 'gy', 'z': 'gz'})
    
    # Ordenar por timestamp
    df_accel = df_accel.sort_values('timestamp').reset_index(drop=True)
    df_gyro = df_gyro.sort_values('timestamp').reset_index(drop=True)
    
    # Merge usando timestamps próximos
    df_combined = pd.merge_asof(
        df_accel, 
        df_gyro, 
        on='timestamp', 
        direction='nearest',
        tolerance=pd.Timedelta(seconds=0.1)
    )
    
    # Remover valores NaN
    df_combined = df_combined.dropna()
    
    return df_combined

## 4. Extração de Features (Janelas Temporais)

In [ ]:
def calcular_features_janela(df, window_size=10):
    """
    Calcula features baseadas em janelas temporais SEM sobreposição.
    Para cada janela, calcula:
    - Variação (desvio padrão) do acelerômetro - FEATURE PRINCIPAL
    - Magnitude média do acelerômetro
    - Variação (desvio padrão) do giroscópio
    - Magnitude média do giroscópio
    
    Args:
        df: DataFrame com dados sincronizados
        window_size: Tamanho da janela (número de pontos)
    
    Returns:
        tuple: (array de features, array de timestamps correspondentes)
    """
    features = []
    timestamps = []
    
    # Janelas SEM sobreposição (step = window_size)
    for i in range(0, len(df) - window_size + 1, window_size):
        window = df.iloc[i:i+window_size]
        
        # Magnitude do acelerômetro
        accel_mag = np.sqrt(window['x']**2 + window['y']**2 + window['z']**2)
        
        # Magnitude do giroscópio
        gyro_mag = np.sqrt(window['gx']**2 + window['gy']**2 + window['gz']**2)
        
        # Features: priorizar VARIAÇÃO sobre magnitude
        feature_vector = [
            accel_mag.std(),       # 1º: Variação acelerômetro (MAIS IMPORTANTE!)
            accel_mag.mean(),      # 2º: Magnitude média acelerômetro
            gyro_mag.std(),        # 3º: Variação giroscópio
            gyro_mag.mean()        # 4º: Magnitude média giroscópio
        ]
        
        features.append(feature_vector)
        
        # Timestamp médio da janela
        ts_medio = window['timestamp'].iloc[len(window)//2]
        timestamps.append(ts_medio)
    
    return np.array(features), np.array(timestamps)

## 5. Aplicação do K-Means

In [ ]:
def aplicar_kmeans(features, n_clusters=3):
    """
    Aplica K-means clustering nas features
    
    Args:
        features: Array de features
        n_clusters: Número de clusters
    
    Returns:
        tuple: (modelo KMeans, labels dos clusters, features normalizadas)
    """
    # Normalizar features
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)
    
    # Aplicar K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features_normalized)
    
    return kmeans, labels, features_normalized

## 6. Mapeamento de Clusters para Rótulos de Movimento

In [ ]:
def map_clusters_to_movement(labels, features):
    """
    Mapeia cada cluster para um rótulo de movimento baseado na VARIAÇÃO (std) do acelerômetro.
    A variação é o melhor indicador: parado tem variação baixa, movimento tem variação alta.
    
    Returns:
        tuple: (dicionário {cluster_id: movimento_str}, array de rótulos por ponto)
    """
    unique_clusters = np.unique(labels)
    cluster_mean_std = {}
    
    # Usar a PRIMEIRA feature (std do acelerômetro) para ordenar
    for c in unique_clusters:
        cluster_mean_std[c] = features[labels == c, 0].mean()
    
    # Ordenar clusters por variação média (ascendente)
    ordered = sorted(cluster_mean_std.items(), key=lambda x: x[1])
    ordered_ids = [c for c, _ in ordered]
    
    # Rótulos para 3 clusters ordenados por variação
    movement_names_3 = [
        'muito baixo movimento (parado)',  # Menor variação
        'baixo movimento',                  # Variação média
        'alto movimento'                    # Maior variação
    ]
    
    if len(ordered_ids) == 3:
        assigned = {cid: movement_names_3[i] for i, cid in enumerate(ordered_ids)}
    else:
        assigned = {cid: f'movimento_{i}' for i, cid in enumerate(ordered_ids)}
    
    labels_movement = np.array([assigned[c] for c in labels])
    
    # Debug: mostrar mapeamento
    print("\n   [DEBUG] Mapeamento por variacao (std acelerometro):")
    for i, (cid, std_val) in enumerate(ordered):
        label = movement_names_3[i] if len(ordered_ids) == 3 else f'movimento_{i}'
        print(f"      Cluster {cid} (std media: {std_val:.4f}) -> {label}")
    
    return assigned, labels_movement

## 7. Visualização dos Resultados

In [ ]:
def plotar_resultados(labels, pessoa_id, timestamps, cluster_name_map, movement_labels):
    """
    Plota os resultados do clustering com timestamps formatados
    
    Args:
        labels: Labels preditos pelo K-means
        pessoa_id: ID da pessoa
        timestamps: Array de timestamps
        cluster_name_map: Dicionário mapeando cluster_id para nome do movimento
        movement_labels: Array de rótulos de movimento por ponto
    """
    fig = plt.figure(figsize=(16, 6))
    
    # Plot 1: Labels ao longo do tempo
    ax1 = plt.subplot(1, 2, 1)
    
    # Definir cores por rótulo de movimento
    palette = {
        'muito baixo movimento (parado)': 'gray',
        'baixo movimento': 'blue',
        'alto movimento': 'red'
    }
    
    default_colors = ['tab:blue', 'tab:orange', 'tab:red', 'gray']
    colors = [palette.get(m, default_colors[i % len(default_colors)]) for i, m in enumerate(movement_labels)]
    
    ax1.scatter(timestamps, movement_labels, c=colors, alpha=0.7, s=40)
    ax1.set_xlabel('Hora (HH:MM:SS)', fontsize=12)
    ax1.set_ylabel('Rotulo de Movimento', fontsize=12)
    ax1.set_title(f'K-means Clustering - Pessoa {pessoa_id}\n(Baseado em variacao - sem sobreposicao)', fontsize=14)
    
    # Formatar eixo X para HH:MM:SS
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=30, ha='right')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Distribuição por rótulo
    ax2 = plt.subplot(1, 2, 2)
    unique_mov, counts = np.unique(movement_labels, return_counts=True)
    mov_colors = [palette.get(m, 'tab:blue') for m in unique_mov]
    ax2.bar(unique_mov, counts, color=mov_colors, alpha=0.7)
    ax2.set_xlabel('Rotulo de Movimento', fontsize=12)
    ax2.set_ylabel('Numero de Janelas', fontsize=12)
    ax2.set_title('Distribuicao por Rotulo de Movimento', fontsize=14)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores nas barras
    for i, count in enumerate(counts):
        ax2.text(i, count, str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    
    # Salvar
    output_dir = '../upload/ClusterK3euclidianoComDownsampling/'
    os.makedirs(output_dir, exist_ok=True)
    output_path = f'{output_dir}clustering_euclidean_pessoa_{pessoa_id}.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n   [OK] Grafico salvo: '{output_path}'")

## 8. Pipeline Completo de Análise

In [ ]:
def analisar_pessoa(pessoa_id, n_clusters=3):
    """
    Análise completa de uma pessoa
    
    Args:
        pessoa_id: ID da pessoa
        n_clusters: Número de clusters
    
    Returns:
        tuple: (dados, features, labels, kmeans, timestamps, movimento)
    """
    print(f"\n{'='*60}")
    print(f"ANALISE - PESSOA {pessoa_id}")
    print(f"{'='*60}")
    
    # 1. Carregar dados
    print("\n[1/5] Carregando dados...")
    df_accel, df_gyro = carregar_dados_pessoa_downsampled(pessoa_id)
    print(f"   [OK] Acelerometro: {len(df_accel)} pontos")
    print(f"   [OK] Giroscopio: {len(df_gyro)} pontos")
    
    # 2. Sincronizar
    print("\n[2/5] Sincronizando dados...")
    df_combined = sincronizar_dados(df_accel, df_gyro)
    print(f"   [OK] Dados sincronizados: {len(df_combined)} pontos")
    
    # 3. Extrair features
    print(f"\n[3/5] Calculando features (janelas SEM sobreposicao)...")
    features, timestamps = calcular_features_janela(df_combined, window_size=WINDOW_SIZE)
    print(f"   [OK] {len(features)} janelas processadas")
    print(f"   [OK] Variacao acelerometro media: {features[:, 0].mean():.4f}")
    
    # 4. Aplicar K-means
    print(f"\n[4/5] Aplicando K-means (k={n_clusters})...")
    kmeans, labels, features_normalized = aplicar_kmeans(features, n_clusters)
    print(f"   [OK] Clustering concluido")
    print(f"   [OK] Inercia: {kmeans.inertia_:.2f}")
    
    # 5. Plotar
    print("\n[5/5] Gerando visualizacao...")
    cluster_map, movement_labels = map_clusters_to_movement(labels, features)
    plotar_resultados(labels, pessoa_id, timestamps, cluster_map, movement_labels)
    
    # Estatísticas
    print(f"\n{'='*60}")
    print("ESTATISTICAS")
    print(f"{'='*60}")
    
    for cluster_id, movimento in cluster_map.items():
        mask = labels == cluster_id
        count = mask.sum()
        std_accel = features[mask, 0].mean()
        mag_accel = features[mask, 1].mean()
        percentage = (count / len(labels)) * 100
        print(f"\n  Cluster {cluster_id} -> {movimento}")
        print(f"     Janelas: {count} ({percentage:.1f}%)")
        print(f"     Variacao acelerometro: {std_accel:.4f}")
        print(f"     Magnitude acelerometro: {mag_accel:.4f}")
    
    return df_combined, features, labels, kmeans, timestamps, movement_labels

## 9. Análise de Todas as Pessoas

In [ ]:
def analisar_todas_pessoas(n_clusters=3):
    """
    Análise de todas as pessoas (11 a 38)
    """
    pessoas = list(range(PESSOA_INICIAL, PESSOA_FINAL + 1))
    resultados = []
    sucessos = 0
    erros = 0
    
    print("\n" + "="*60)
    print("ANALISE DE TODAS AS PESSOAS")
    print("="*60)
    
    for pessoa_id in pessoas:
        try:
            df_combined, features, labels, kmeans, timestamps, movimento = analisar_pessoa(
                pessoa_id, n_clusters
            )
            resultados.append({
                'pessoa_id': pessoa_id,
                'data': df_combined,
                'features': features,
                'labels': labels,
                'kmeans': kmeans,
                'timestamps': timestamps,
                'movimento': movimento
            })
            sucessos += 1
        except Exception as e:
            print(f"\n[ERRO] Pessoa {pessoa_id}: {e}")
            erros += 1
    
    # Estatísticas finais
    print("\n" + "="*60)
    print("ESTATISTICAS FINAIS")
    print("="*60)
    print(f"Total processadas: {len(resultados)}/{len(pessoas)}")
    print(f"Sucessos: {sucessos} | Erros: {erros}")
    
    if resultados:
        # Distribuição geral
        dist_geral = {
            'muito baixo movimento (parado)': 0,
            'baixo movimento': 0,
            'alto movimento': 0
        }
        
        total_janelas = 0
        for r in resultados:
            movimento = r['movimento']
            unique, counts = np.unique(movimento, return_counts=True)
            for mov, count in zip(unique, counts):
                if mov in dist_geral:
                    dist_geral[mov] += count
                total_janelas += count
        
        # Normalizar
        for k in dist_geral:
            dist_geral[k] = dist_geral[k] / total_janelas if total_janelas > 0 else 0
        
        print("\nDISTRIBUICAO GERAL:")
        print(f"   [PARADO] Muito baixo movimento: {dist_geral['muito baixo movimento (parado)']:.1%}")
        print(f"   [BAIXO] Baixo movimento: {dist_geral['baixo movimento']:.1%}")
        print(f"   [ALTO] Alto movimento: {dist_geral['alto movimento']:.1%}")
    
    return resultados

## 10. Executar Análise

**Escolha uma das opções abaixo:**

### Opção A: Analisar UMA pessoa específica

In [ ]:
# Analisar pessoa específica
pessoa_escolhida = 11

df, features, labels, kmeans, timestamps, movimento = analisar_pessoa(
    pessoa_escolhida, 
    n_clusters=N_CLUSTERS
)

### Opção B: Analisar TODAS as pessoas (11-38)

In [ ]:
# Analisar todas as pessoas
resultados = analisar_todas_pessoas(n_clusters=N_CLUSTERS)

print("\n" + "="*60)
print("ANALISE CONCLUIDA!")
print("="*60)
print(f"Total processadas: {len(resultados)}")
print(f"Graficos salvos em: upload/ClusterK3euclidianoComDownsampling/")

## 11. Análises Adicionais (Opcional)

### Visualizar distribuição de uma pessoa específica

In [ ]:
# Exemplo: visualizar distribuição detalhada
if 'resultados' in locals():
    pessoa_exemplo = resultados[0]
    
    movimento = pessoa_exemplo['movimento']
    features = pessoa_exemplo['features']
    
    print(f"\nPessoa {pessoa_exemplo['pessoa_id']}:")
    print(f"Total de janelas: {len(movimento)}")
    
    for mov_type in np.unique(movimento):
        mask = movimento == mov_type
        count = mask.sum()
        std_medio = features[mask, 0].mean()
        print(f"  {mov_type}: {count} janelas (std medio: {std_medio:.4f})")